#Step 1 .Establish the data connection between my Azure Data Lake Storage (ADLS) and Data Factory.

In [0]:
# Configuration Template 
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "[client_id]",
"fs.azure.account.oauth2.client.secret": "[secret_id]",
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/[tenent_id]/oauth2/token"}

# Mounting point 
dbutils.fs.mount(
source = "abfss://[container-name]@[storage account name].dfs.core.windows.net", 
mount_point = "/mnt/[some name you want to mount it as]",  #i called it "homerental"
extra_configs = configs)

Actual configuration code is hidden.

In [0]:
#check if the entire data lake is mounted to this location
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/homerental,abfss://home-rental-data@homerentalanalysis.dfs.core.windows.net,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,
/volume,DbfsReserved,


In [0]:
# in Databricks, you don't have to create a Spark session from scatch 
# such as from pyspark.sql import SparkSession
spark

#Step 2. Data Transformation

In [0]:
# write Spark code to read files
capacity = spark.read.format("csv").option("header","true").load("/mnt/homerental/raw-data/capacity.csv")
pricing = spark.read.format("csv").option("header","true").load("/mnt/homerental/raw-data/pricing.csv")
applicant = spark.read.format("csv").option("header","true").load("/mnt/homerental/raw-data/applicant.csv")

In [0]:
# display table output
capacity.show()

+-------------------+----------+--------+
| Full building name|Coded name|Capacity|
+-------------------+----------+--------+
| Amazing Apartment |     APT-1|     200|
|Brilliant Apartment|     APT-3|     300|
|Fantastic Apartment|     APT-5|     400|
|      Great College| College-1|     300|
| Incredible College| College-3|     500|
|  Wonderful College| College-5|     500|
+-------------------+----------+--------+



In [0]:
# display table output
pricing.show()

+-------------+-----------+------------+-----------------+------------------+
|Room Building|  Room Type|Monthly Rent|Monthly Utilities|Total Monthly Cost|
+-------------+-----------+------------+-----------------+------------------+
|        APT-1|Single Room|        1000|               50|              1050|
|        APT-1|  1 Bedroom|         950|               50|              1000|
|        APT-1|  2 Bedroom|         900|               50|               950|
|        APT-1|  5 Bedroom|         800|               50|               850|
|        APT-3|Single Room|        2000|              100|              2100|
|        APT-3|  1 Bedroom|        1500|              100|              1600|
|        APT-3|  2 Bedroom|        1000|              100|              1100|
|        APT-3|  5 Bedroom|         900|              100|              1000|
|        APT-5|Single Room|        1500|              120|              1620|
|        APT-5|  1 Bedroom|        1200|              120|      

In [0]:
# display table output
applicant.show()

+----+--------+-----------------+-------------+--------------+------------+----------------+-------------------+--------------------------+----------------+---------------+----------------------+---------------------+--------------------+-------------------+-----------------+---------------------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------+
| Age|Entry ID|           Gender|         City|State Province|Zip Postcode|Application Date|Application Status |Classification Description|Offer Reply Date|Offer Sent Date|Apply for Scholarship?|Children accompanied?|Partner accompanied?|   Location offered|Room Type offered|Aboriginal/Torres Strait Islander|Course Description|Enrollment Class| 1st Room Preference| 2nd Room Preference| 3rd Room Preference| Student Type|
+----+--------+-----------------+-------------+--------------+------------+----------------+-------------------+--------------------------+-------------

In [0]:
# printSchema checks the data type of each column 
# Helpful to check if anything is wrong in the data frame, like wrong data type 
# Do this check for each data frame 
capacity.printSchema()

# "Capacity" column should be integer instead of string

root
 |-- Full building name: string (nullable = true)
 |-- Coded name: string (nullable = true)
 |-- Capacity: string (nullable = true)



In [0]:
pricing.printSchema()

# "Monthly Rent", "Monthly Utilities", and "Total Monthly Cost" columns should be integer instead of string

root
 |-- Room Building: string (nullable = true)
 |-- Room Type: string (nullable = true)
 |-- Monthly Rent: string (nullable = true)
 |-- Monthly Utilities: string (nullable = true)
 |-- Total Monthly Cost: string (nullable = true)



In [0]:
applicant.printSchema()

# "Age", "Entry ID" columns should be integer instead of string 
# "Application Date", "Offer Reply Date", "Offer Sent Date" columns should be date instead of string
# "Apply for Scholarship?"", "Children accompanied?"", "Partner accompanied?"" columns should be bool instead of string

root
 |-- Age: string (nullable = true)
 |-- Entry ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State Province: string (nullable = true)
 |-- Zip Postcode: string (nullable = true)
 |-- Application Date: string (nullable = true)
 |-- Application Status : string (nullable = true)
 |-- Classification Description: string (nullable = true)
 |-- Offer Reply Date: string (nullable = true)
 |-- Offer Sent Date: string (nullable = true)
 |-- Apply for Scholarship?: string (nullable = true)
 |-- Children accompanied?: string (nullable = true)
 |-- Partner accompanied?: string (nullable = true)
 |-- Location offered: string (nullable = true)
 |-- Room Type offered: string (nullable = true)
 |-- Aboriginal/Torres Strait Islander: string (nullable = true)
 |-- Course Description: string (nullable = true)
 |-- Enrollment Class: string (nullable = true)
 |-- 1st Room Preference: string (nullable = true)
 |-- 2nd Room Preference: string (

Change data type method 1: using withColumn

In [0]:
from pyspark.sql.functions import col 
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
# Change data type from string to integer 
# Create a new column in the right data format that is overwriting the old column in the wrong data format 

# From capacity data frame 
# "Capacity" column should be integer instead of string
capacity = capacity.withColumn("Capacity", col("Capacity").cast(IntegerType()))

# From pricing data frame 
# "Monthly Rent", "Monthly Utilities", and "Total Monthly Cost" columns should be integer instead of string.
pricing = pricing.withColumn("Monthly Rent", col("Monthly Rent").cast(IntegerType()))\
    .withColumn("Monthly Utilities", col("Monthly Utilities").cast(IntegerType()))\
    .withColumn("Total Monthly Cost", col("Total Monthly Cost").cast(IntegerType()))

# From applicant data frame 
# "Age", "Entry ID" columns should be integer instead of string 
applicant = applicant.withColumn("Age", col("Age").cast(IntegerType()))\
    .withColumn("Entry ID", col("Entry ID").cast(IntegerType()))

In [0]:
# print schema to confirm successful data type switch 
capacity.printSchema()

root
 |-- Full building name: string (nullable = true)
 |-- Coded name: string (nullable = true)
 |-- Capacity: integer (nullable = true)



Change data type method 2: using inferSchema during the file reading step

In [0]:
applicant = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("/mnt/homerental/raw-data/applicant.csv")

In [0]:
applicant.printSchema()

# most columns are correctly interpreted as boolean and date data types, except "Application Date" column

root
 |-- Age: double (nullable = true)
 |-- Entry ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State Province: string (nullable = true)
 |-- Zip Postcode: double (nullable = true)
 |-- Application Date: string (nullable = true)
 |-- Application Status : string (nullable = true)
 |-- Classification Description: string (nullable = true)
 |-- Offer Reply Date: date (nullable = true)
 |-- Offer Sent Date: date (nullable = true)
 |-- Apply for Scholarship?: boolean (nullable = true)
 |-- Children accompanied?: boolean (nullable = true)
 |-- Partner accompanied?: boolean (nullable = true)
 |-- Location offered: string (nullable = true)
 |-- Room Type offered: string (nullable = true)
 |-- Aboriginal/Torres Strait Islander: string (nullable = true)
 |-- Course Description: string (nullable = true)
 |-- Enrollment Class: string (nullable = true)
 |-- 1st Room Preference: string (nullable = true)
 |-- 2nd Room Preference: string (

In [0]:
# From applicant data frame 
# "Age", "Entry ID" columns should be integer instead of string 
applicant = applicant.withColumn("Application Date", col("Application Date").cast(DateType()))

In [0]:
applicant.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Entry ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State Province: string (nullable = true)
 |-- Zip Postcode: double (nullable = true)
 |-- Application Date: date (nullable = true)
 |-- Application Status : string (nullable = true)
 |-- Classification Description: string (nullable = true)
 |-- Offer Reply Date: date (nullable = true)
 |-- Offer Sent Date: date (nullable = true)
 |-- Apply for Scholarship?: boolean (nullable = true)
 |-- Children accompanied?: boolean (nullable = true)
 |-- Partner accompanied?: boolean (nullable = true)
 |-- Location offered: string (nullable = true)
 |-- Room Type offered: string (nullable = true)
 |-- Aboriginal/Torres Strait Islander: string (nullable = true)
 |-- Course Description: string (nullable = true)
 |-- Enrollment Class: string (nullable = true)
 |-- 1st Room Preference: string (nullable = true)
 |-- 2nd Room Preference: string (nu

#Step 3. Data Analysis

In [0]:
# Question: Find the housing option with the highest monthly utilities
# If you want to show all columns in the table 
highest_utilities_housing = pricing.orderBy("Monthly Utilities", ascending=False).show()

+-------------+-----------+------------+-----------------+------------------+
|Room Building|  Room Type|Monthly Rent|Monthly Utilities|Total Monthly Cost|
+-------------+-----------+------------+-----------------+------------------+
|        APT-5|  2 Bedroom|        1050|              120|              1170|
|    College-5|Single Room|        1300|              120|              1420|
|        APT-5|  5 Bedroom|        1000|              120|              1120|
|        APT-5|Single Room|        1500|              120|              1620|
|        APT-5|  1 Bedroom|        1200|              120|              1320|
|    College-5|  1 Bedroom|        1200|              120|              1320|
|    College-5|  5 Bedroom|        1000|              120|              1120|
|    College-5|  2 Bedroom|        1050|              120|              1170|
|        APT-3|  2 Bedroom|        1000|              100|              1100|
|        APT-3|Single Room|        2000|              100|      

In [0]:
# If you want to show only selected columns in the table 
highest_utilities_housing = pricing.orderBy("Monthly Utilities", ascending=False).select("Room Building", "Room Type", "Monthly Utilities").show()

+-------------+-----------+-----------------+
|Room Building|  Room Type|Monthly Utilities|
+-------------+-----------+-----------------+
|        APT-5|  2 Bedroom|              120|
|    College-5|Single Room|              120|
|        APT-5|  5 Bedroom|              120|
|        APT-5|Single Room|              120|
|        APT-5|  1 Bedroom|              120|
|    College-5|  1 Bedroom|              120|
|    College-5|  5 Bedroom|              120|
|    College-5|  2 Bedroom|              120|
|        APT-3|  2 Bedroom|              100|
|        APT-3|Single Room|              100|
|    College-3|  2 Bedroom|              100|
|    College-3|Single Room|              100|
|        APT-3|  5 Bedroom|              100|
|    College-3|  5 Bedroom|              100|
|        APT-3|  1 Bedroom|              100|
|    College-3|  1 Bedroom|              100|
|    College-1|Single Room|               50|
|        APT-1|  1 Bedroom|               50|
|        APT-1|  5 Bedroom|       

# Step 4. Output the transformed dataframes into csv format

-It will be in a designated file folder located under the mounting point

-There are 3 templates, depending on whether it is your first time writing on the output csv file, and whether you would like to partition the csv file into multiple files (because csv file is too large to be in one).

In [0]:
# Template: output the transformed dataframe to csv in transformed-data file folder
[dataframe_name].write.option("header","true").csv("/mnt/homerental/transformed-data/dataframe_name")

In [0]:
# Template: if want to overwrite on the same file
[dataframe_name].write.mode(“overwrite”).option("header","true").csv("/mnt/[mounting_point_name]/[folder_name]/[dataframe_name]")

In [0]:
# Template: If the output csv is too big, you can split the output csv into n-many files.
[dataframe_name].repartition([n])write.mode(“overwrite”).option("header","true").csv("/mnt/[mounting_point_name]/[folder_name]/[dataframe_name]")


I am going to export all of my csv files in the default format (no partitioning nor overwriting).

In [0]:
applicant.write.option("header","true").csv("/mnt/homerental/transformed-data/applicant")


In [0]:
capacity.write.option("header","true").csv("/mnt/homerental/transformed-data/capacity")

In [0]:
pricing.write.option("header","true").csv("/mnt/homerental/transformed-data/pricing")